In [2]:
import sympy as sp
from sympy import Matrix as spmtx
from sympy import symbols as syms
import math
from collections import deque
import copy
import random

### Monomials
Lets start with monomials

In [3]:
def init_mono(var, degree, coeff):
    if len(coeff) != degree + 1:
        raise Exception("Degree coeff doesn't match")
        return
    poly = coeff[0]
    base = [1]
    for i in range(1, degree+1):
        base.append(var**i)
    return base, coeff

In [4]:
x = syms('x')
b, c = init_mono(x, 4, [2, 3, 5, 7, 1])
b

[1, x, x**2, x**3, x**4]

In [5]:
def init_scaled_mono_basis(var, degree, poly):
    p = 1
    mb, c = poly
    for i in range(1, degree+1):
        p *= math.factorial(i)
    const = math.factorial(degree) / p
    base = [1]
    coeff = [c[0]/p]
    for i in range(1, degree+1):
        base.append(var**i)
        coeff.append(c[i] / p)
    return base, coeff

In [6]:
init_scaled_mono_basis(x, 4, (b, c))

([1, x, x**2, x**3, x**4],
 [0.006944444444444444,
  0.010416666666666666,
  0.017361111111111112,
  0.024305555555555556,
  0.003472222222222222])

# Multivariate Polynomial

### Initalize a polynomial using monomial bases

##### General Procedure

This is some text that I am writing so that I know what I am doing, and I can check my understand with Prof.Dressler. 

Goal: Condition number of a matrix $A$, but what is this matrx.

When fomulazing the SDP, we want to solve $b^T Q b = p(x)$ with some $Q$ positive semidefinte. There are well known algroithms so I don't really need to deal with it.

However, what we are interested in is the formulazation of $b^T Q b = p(x)$.

In particular, let $Q = \begin{pmatrix} q_{11} &  q_{12} & q_{13} \\ q_{12} & q_{22} & q_{23} \\ q_{13} & q_{23} & q_{33} \end{pmatrix}$ we can constrant this by the matching of coefficient method. 

Therefore, given a basis of the polynomial with half of the total degree, we actually have a linear system $Aq = b$ where $b$ is the coefficient of the polynomial expressed in the given basis. And we are interested in the condition number of $A$ 

In [2]:
def init_poly(var_num, degree, coeff=None):
    '''
    Init a multivariate polynomial with given num of var and num of degree.
    The base used here is monomial
    '''
    var_list = deque()
    for i in range(var_num):
        var_list.append(syms('x_' + str(i)))
    terms = []
    for i in range(degree, -1, -1):
        terms += gen_terms(var_list, i, 1)
    if coeff == None:
        coeff = []
        for i in range(len(terms)):
            coeff.append(random.randint(-100, 100))
    return var_list, terms, coeff

In [3]:
def gen_terms(var_list, total_degree, term):
    '''
    var_list is of type deque
    '''
    terms = []
    if len(var_list) == 1:
        term *= var_list[0] ** total_degree
        terms.append(term)
        return terms
    elif total_degree == 0:
        term *= 1
        terms.append(term)
        return terms
    else:
        for i in range(total_degree, -1, -1):
            var_list_further = copy.deepcopy(var_list)
            var = var_list_further.popleft()
            term_further = term * var ** i
            terms += gen_terms(var_list_further, total_degree - i, term_further)
        return terms

In [56]:
v, d, c = init_poly(2, 2, coeff=None)

In [64]:
d # lexicographic order or reveres lexicographic order

[x_0**2, x_0*x_1, x_1**2, x_0, x_1, 1]

In [58]:
c

[-33, -63, -93, 94, 24, -5]

### Convert to Quadratic form

Given a polynomial (terms, coeffs), and a bases (basis), convert it into the form $b^TQb$, where Q is a matrix. 
We want to generate the linear equations that constraint Q
- Q should be a symmetric matrix

In [62]:
basis = init_poly(2, 1, coeff=None)[1]

In [63]:
basis

[x_0, x_1, 1]

In [59]:
basis = [1, v[0], v[1]]

In [65]:
def gen_monomial_basis(num_of_var, degree):
    var_list = deque()
    for i in range(num_of_var):
        var_list.append(syms('x_' + str(i)))
    terms = []
    for i in range(degree, -1, -1):
        terms += gen_terms(var_list, i, 1)
    return terms

In [66]:
def gen_linear_system(basis, terms, coeffs):
    Q_dim = len(basis)
    Q = mtx.zeros(Q_dim)
    mtx_var = set()
    for i in range(Q_dim):
        for j in range(i, Q_dim):
            Q[i, j] = (syms('q_(' + str(i) + ')(' + str(j) + ')'))
            Q[j, i] = (syms('q_(' + str(i) + ')(' + str(j) + ')'))
            mtx_var.add(syms('q_(' + str(i) + ')(' + str(j) + ')'))
    quad = basis.T * Q * basis
    quad = sp.expand(quad)
    r = sp.collect(quad[0], (syms('x_0'), syms('x_1')))
    coeff = dict()
    for b in basis:
        print(b)
        args = quad[0].coeff(b).args
        coeff[b] = 0
        for arg in args:
            not_in = True
            for b_itr in basis:
                if b_itr in arg.free_symbols:
                    not_in = False
                    break
            if not_in == True:
                coeff[b] += arg
        print(coeff[b])
        print('---------')
    coeff_list = []
    for key in coeff:
        coeff_list.append(coeff[key])
    A, b = sp.linear_eq_to_matrix(coeff_list, list(mtx_var))
    return A, b

In [67]:
basis = gen_monomial_basis(2, 1)
basis = mtx(basis)

In [68]:
basis

Matrix([
[x_0],
[x_1],
[  1]])

In [69]:
A, b = gen_linear_system(basis, d, c)

x_0
2*q_(0)(2)
---------
x_1
2*q_(1)(2)
---------
1
q_(2)(2)
---------


In [70]:
def condition_num(A, norm = None):
    min_dim = min(A.shape)
    if A.rank() != min_dim:
        return -1 # since condition number is geq 1, this indicate it is not full rank
    else:
        cond = A.pinv().norm() * A.norm()
        return cond

In [71]:
condition_num(A)

3*sqrt(6)/2

In [18]:
Q_dim = len(basis)
Q = mtx.zeros(Q_dim)
mtx_var = set()
for i in range(Q_dim):
    for j in range(i, Q_dim):
        Q[i, j] = (syms('q_(' + str(i) + ')(' + str(j) + ')'))
        Q[j, i] = (syms('q_(' + str(i) + ')(' + str(j) + ')'))
        mtx_var.add(syms('q_(' + str(i) + ')(' + str(j) + ')'))

In [19]:
list(mtx_var)

[q_(2)(2), q_(0)(1), q_(0)(0), q_(0)(2), q_(1)(2), q_(1)(1)]

In [20]:
Q

Matrix([
[q_(0)(0), q_(0)(1), q_(0)(2)],
[q_(0)(1), q_(1)(1), q_(1)(2)],
[q_(0)(2), q_(1)(2), q_(2)(2)]])

In [21]:
quad = basis.T * Q * basis
quad = sp.expand(quad)
r = sp.collect(quad[0], (syms('x_0'), syms('x_1')))
A = mtx.zeros(len(basis))
coeff = dict()
for b in basis:
    print(b)
    args = quad[0].coeff(b).args
    coeff[b] = 0
    for arg in args:
        not_in = True
        for b_itr in basis:
            if b_itr in arg.free_symbols:
                not_in = False
                break
        if not_in == True:
            coeff[b] += arg
    print(coeff[b])
    print('---------')


x_0
2*q_(0)(2)
---------
x_1
2*q_(1)(2)
---------
1
q_(2)(2)
---------


#### Here should be some experiments to get a statsitical behavior of the condition number

In [22]:

coeff_list = []
for key in coeff:
    coeff_list.append(coeff[key])

In [23]:
coeff_list

[2*q_(0)(2), 2*q_(1)(2), q_(2)(2)]

In [24]:
A, b = sp.linear_eq_to_matrix(coeff_list, list(mtx_var))

In [25]:
A.condition_number()

zoo

In [26]:
A.rank()

3

In [27]:
A.pinv().norm() * A.norm()

3*sqrt(6)/2

In [111]:
quad

Matrix([[q_(0)(0)*x_0**2 + 2*q_(0)(1)*x_0*x_1 + 2*q_(0)(2)*x_0 + q_(1)(1)*x_1**2 + 2*q_(1)(2)*x_1 + q_(2)(2)]])

In [102]:
syms('x_0') in args[0].free_symbols

False

### Scaled Monomials

In [6]:
a = syms('a')
b = syms('b')

sp.expand((a+b)**3 - (a-b)**3)

6*a**2*b + 2*b**3

In [ ]:
def gen_basis(num_of_var, degree):
    basis = []
    basis.append()